In [1]:
import numpy as np
import networkx as nx
import random
import matplotlib.pyplot as plt

In [2]:
class Agent:
    
    def __init__(self, id_number, sma, accointances, stats): 
        self.id_number = id_number
        self.accointances = accointances
        self.sma = sma
        self.extra = stats["extra"]
        self.travail = stats["travail"]
        self.mature = stats["mature"]
        self.subjectif = stats["subjectif"]
        self.interet = False
        self.Dead = False
        
    def getVoisins(self):
        return [self.sma.getAgent(i) for i in self.accointances]
    
    def decide(self, tick):
        if not(self.Dead):
            voisins = self.getVoisins()
            if self.subjectif:
                [self.giveInteretTo(voisin) for voisin in voisins if self == voisin]
            else:
                [self.giveInteretTo(voisin) for voisin in voisins if self != voisin]
                
    def giveInteretTo(self,voisin):
        voisin.interet = True
     
    def __eq__(self, o):
        return isinstance(o,Agent) and self.extra == o.extra and self.travail == o.travail and self.mature == self.mature
    
    def __str__(self):
        return "- Agent "+str(self.id_number)+" -"
    
    def __repr__(self):
        return str(self)            

In [3]:
class SMA:
    
    def __init__(self, nb_agents,graph,adjacency_matrix,pbE,pbT,pbM,pbS):
        self.tick = 0
        self.nb_agents = nb_agents
        self.agentList = []
        self.graph = graph
        self.pbs = [pbE,pbT,pbM,pbS]
        for i in range(nb_agents):
            self.agentList.append(self.createAgent(i,adjacency_matrix))
            
    def createAgent(self,i,adjacency_matrix):
        stats = dict()
        r = random.random()
        statNames = [("extra",self.pbs[0]),("travail",self.pbs[1]),("mature",self.pbs[2]),("subjectif",self.pbs[3])]
        for stat in statNames:
            stats[stat[0]] = (r < stat[1])
        return Agent(i, self, np.nonzero(adjacency_matrix[i])[0],stats)
    
    def getAgent(self,i):
        return self.agentList[i]
    
    def createSMA_graph(graph,pbE,pbT,pbM,pbS):
        matrix = nx.to_numpy_array(graph, nodelist=range(graph.number_of_nodes()))
        return SMA(graph.number_of_nodes(),graph, matrix, pbE,pbT,pbM,pbS)
    
    def createSMA_nbagents(nb_agents,nb_contact,pbE,pbT,pbM,pbS):
        graph = nx.barabasi_albert_graph(nb_agents, nb_contact)
        return SMA.createSMA_graph(graph,pbE,pbT,pbM,pbS)
    
    def run(self, rounds):
        self.tick = 0
        self.clearAll()
        oldNbLives = -1
        newNbLives = self.nbLives()
        while self.tick < rounds and oldNbLives != newNbLives and not(self.allDead()):
            oldNbLives = newNbLives
            self.runOnce()
            newNbLives = self.nbLives()
        return self.nbLives()
            
    def runOnce(self):
        self.tick += 1
        self.setAll()
        for agent in self.agentList:
            agent.decide(self.tick)
        self.setDeath()
            
    def allDead(self):
        return all([agent.Dead for agent in self.agentList])
    
    def setAll(self):
        for agent in self.agentList:
            agent.interet = False
            
    def setDeath(self):
        for agent in self.agentList:
            if not(agent.interet):
                agent.Dead = True
    
    def clearAll(self):
        for agent in self.agentList:
            agent.interet = False
            agent.Dead = False
            
    def nbLives(self):
        res = 0
        for agent in self.agentList:
            if not(agent.Dead):
                res += 1
        return res

In [4]:
from deap import creator, tools, base, algorithms

def evaluate_SMA(N,nb_contact,ind):
    res = []
    for i in range(100):
        sma = SMA.createSMA_nbagents(N,nb_contact,ind[0],ind[1],ind[2],ind[3])
        res += [sma.run(100)]
    return (np.mean(res),)
    

def algo_gen_SMA(nb_ind_pop,nb_gen,nb_contact):
    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMax)
    
    toolbox=base.Toolbox()
    toolbox.register("att",random.random)
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.att, 4)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("evaluate",evaluate_SMA,nb_ind_pop,nb_contact)
    toolbox.register("mutate",tools.mutShuffleIndexes,indpb=0.2)
    toolbox.register("mate",tools.cxOnePoint)
    toolbox.register("select",tools.selBest)
    
    hof = tools.HallOfFame(1)

    stats = tools.Statistics(lambda ind : ind.fitness.values)
    stats.register("avg",np.mean)
    stats.register("std",np.std)
    stats.register("min",np.min)
    stats.register("max",np.max)
    
    p = toolbox.population(n=nb_ind_pop)
    
    pop, log = algorithms.eaSimple(p, toolbox,cxpb=0.5, mutpb=0.2 , ngen=nb_gen , stats=stats, halloffame=hof, verbose=False)
    # pop, log = algorithms.eaMuPlusLambda(p, toolbox, mu=len(p), lambda_=len(p)//2,cxpb=0.5, mutpb=0.2 , ngen=nb_gen , stats=stats, halloffame=hof, verbose=False)
    
    return {"pop":pop,"log":log,"hof":hof,"stats": stats}

In [33]:
N = 10
N_gen = 100
N_contact = 3

exemples = [algo_gen_SMA(N,N_gen,N_contact) for _ in range(5)]

In [36]:
for i in range(5):
    print(exemples[i]["hof"][0])

[0.9885790440181539, 0.9863487964155002, 0.11439252878648731, 0.9289995660879429]
[0.9421081837563204, 0.08395343157452273, 0.9395278585769126, 0.9771571517341212]
[0.014014248535267337, 0.999929560455892, 0.9702449823412255, 0.9326979768187923]
[0.9752475982427464, 0.9601164411677091, 0.9636533632286904, 0.9015081500093076]
[0.41621536907077994, 0.5149339882652899, 0.5785749838237019, 0.00132349420690128]


In [35]:
exemples = [
    {"hof":[[0.9885790440181539, 0.9863487964155002, 0.11439252878648731, 0.9289995660879429]]},
    {"hof":[[0.9421081837563204, 0.08395343157452273, 0.9395278585769126, 0.9771571517341212]]},
    {"hof":[[0.014014248535267337, 0.999929560455892, 0.9702449823412255, 0.9326979768187923]]},
    {"hof":[[0.9752475982427464, 0.9601164411677091, 0.9636533632286904, 0.9015081500093076]]},
    {"hof":[[0.41621536907077994, 0.5149339882652899, 0.5785749838237019, 0.00132349420690128]]}
]

In [ ]:
best = a["hof"][0]
best.fitness.value = evaluate_SMA(100,3,best)
print("best  :  "+ str(best) +" : "+str(best.fitness.value))

for i in a["pop"]:
   i.fitness.value = evaluate_SMA(100,3,i)
   print(str(i)+ " : " + str(i.fitness.value))
    
print()
print(str(evaluate_SMA(100,8,[0.5,0.5,0.5,1])))
print(str(evaluate_SMA(100,3,[0,0,0,1])))
print(str(evaluate_SMA(100,3,[1,0,0,1])))
print(str(evaluate_SMA(100,3,[0,1,0,1])))
